In [1]:
import pandas as pd
import sys

import nbimporter

import Shading_Correction_Factor_Main as SCFMain

Importing Jupyter notebook from Shading_Correction_Factor_Main.ipynb
Importing Jupyter notebook from climate.ipynb
Importing Jupyter notebook from sun_position.ipynb
Importing Jupyter notebook from effect_coefficient.ipynb
Importing Jupyter notebook from direct_solar_area.ipynb
Importing Jupyter notebook from shading_effect_factor.ipynb


# 日よけ効果係数算定ツール  Calculation

- 一次目標：日よけ効果係数算定ツールのpython上での再現


## X. 日よけ効果係数算定ツール本体＋入出力



### X.3 計算実施

#### X.3.1 \日除け効果係数算定シート(全対応)-9.xlsm の「算定」シート セルEC8～FD29 全96ケースを計算

In [2]:
""" 全96ケース計算 """
# \確認.xlsx "本体テスト1"シート → \AllTest01.csv を作成 → \TestConfig01 下に置いて読み込み → 計算

csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/AllTest01.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="AllTest01_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Path00, FileName00, FileName01, ClimateZone, NDT, etaID, Azimuth] \
        = csv_input.values[i][0:8]
    WSSize = csv_input.values[i][8:-2]
    Azimuth = {
        'S' : '南',
        'SW': '南西',
        'W' : '西',
        'NW': '北西',
        'N' : '北',
        'NE': '北東',
        'E' : '東',
        'SE': '南東'
    }[Azimuth]

    [SCFhA, SCFcA] = csv_input.values[i][-2:]
    SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                               , ClimateZone, NDT, etaID, Azimuth, WSSize )
        # SCF[h][i][j]
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算             
    [SCFh, SCFc] = [SCF[2][-1][-1], SCF[2][-2][-1]]
    print('case = {}: SCFh = {}, SCFc = {}'.format( case, SCFh, SCFc ))

    with open("./SCFCalc01/AllTest01.csv","a") as outputfile:
        for h in range(0,3,1):
            print([[SCF[0][ii][j] for j in range(-1,25,1)] for ii in range(-2,13)], file=outputfile)
    

case = 1: SCFh = 0.8014719633667962, SCFc = 0.6890803219314844
case = 2: SCFh = 0.8065830554110296, SCFc = 0.7235658022869215
case = 3: SCFh = 0.8260363713287406, SCFc = 0.7697963294245972
case = 4: SCFh = 0.8179696023209658, SCFc = 0.7968673660374885
case = 5: SCFh = 0.8063919169335321, SCFc = 0.8069052665640057
case = 6: SCFh = 0.8167106493328777, SCFc = 0.7955289052309528
case = 7: SCFh = 0.8231561020450423, SCFc = 0.7645089801266521
case = 8: SCFh = 0.8070115655981551, SCFc = 0.7177709275875591
case = 9: SCFh = 0.791377241752669, SCFc = 0.6675762471056538
case = 10: SCFh = 0.7936937005704867, SCFc = 0.7037783732350965
case = 11: SCFh = 0.8040750971914564, SCFc = 0.7482415709371907
case = 12: SCFh = 0.7785895266406063, SCFc = 0.7678191700776748
case = 13: SCFh = 0.7626110823894359, SCFc = 0.7687368373994354
case = 14: SCFh = 0.7770925063448118, SCFc = 0.764461081568263
case = 15: SCFh = 0.8019823224361669, SCFc = 0.7409225269791503
case = 16: SCFh = 0.7945784700184667, SCFc = 0.6966

#### X.3.2 \日除け効果係数算定シート(全対応)-9.xlsm の「算定」シート セルEC8～FD29 全96ケースを計算
- ただし、入射角特性非考慮の場合の検討

In [3]:
""" 全96ケース計算 - 入射角特性非考慮の場合 """
# \確認.xlsx "本体テスト1"シート → \AllTest01.csv を作成 → \TestConfig01 下に置いて読み込み → 計算
# ●●● ID=0：日よけ効果係数内で入射角特性非考慮 ●●●

csv_input = pd.read_csv(filepath_or_buffer="./TestConfig01/AllTest01.csv", encoding="ms932", sep=",")
if csv_input.columns[0]!="AllTest01_case":
    sys.exit("ファイル内に貼り付けたテスト条件が違います")
for i in range(len(csv_input)):
    [case, Path00, FileName00, FileName01, ClimateZone, NDT, etaID, Azimuth] \
        = csv_input.values[i][0:8]
    WSSize = csv_input.values[i][8:-2]
    Azimuth = {
        'S' : '南',
        'SW': '南西',
        'W' : '西',
        'NW': '北西',
        'N' : '北',
        'NE': '北東',
        'E' : '東',
        'SE': '南東'
    }[Azimuth]
    [SCFhA, SCFcA] = csv_input.values[i][-2:]
    #
    etaID = 0   # 入射角特性をここでハック
    #
    SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                               , ClimateZone, NDT, etaID, Azimuth, WSSize )
        # SCF[h][i][j]
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算             
    [SCFh, SCFc] = [SCF[2][-1][-1], SCF[2][-2][-1]]
    print('case = {}: SCFh = {}, SCFc = {}'.format( case, SCFh, SCFc ))

    with open("./SCFCalc01/AllTest01n.csv","a") as outputfile:
        for h in range(0,3,1):
            print([[SCF[0][ii][j] for j in range(-1,25,1)] for ii in range(-2,13)], file=outputfile)

case = 1: SCFh = 0.8014719633667962, SCFc = 0.6890803219314844
case = 2: SCFh = 0.8065830554110296, SCFc = 0.7235658022869215
case = 3: SCFh = 0.8260363713287406, SCFc = 0.7697963294245972
case = 4: SCFh = 0.8179696023209658, SCFc = 0.7968673660374885
case = 5: SCFh = 0.8063919169335321, SCFc = 0.8069052665640057
case = 6: SCFh = 0.8167106493328777, SCFc = 0.7955289052309528
case = 7: SCFh = 0.8231561020450423, SCFc = 0.7645089801266521
case = 8: SCFh = 0.8070115655981551, SCFc = 0.7177709275875591
case = 9: SCFh = 0.791377241752669, SCFc = 0.6675762471056538
case = 10: SCFh = 0.7936937005704867, SCFc = 0.7037783732350965
case = 11: SCFh = 0.8040750971914564, SCFc = 0.7482415709371907
case = 12: SCFh = 0.7785895266406063, SCFc = 0.7678191700776748
case = 13: SCFh = 0.7626110823894359, SCFc = 0.7687368373994354
case = 14: SCFh = 0.7770925063448118, SCFc = 0.764461081568263
case = 15: SCFh = 0.8019823224361669, SCFc = 0.7409225269791503
case = 16: SCFh = 0.7945784700184667, SCFc = 0.6966

#### X.3.3「デフォルト値」確認試算

In [4]:
""" 「デフォルト値」確認試算 """


Path00 = "./SCFConfig01/"  # 設定ファイルのあるパス
FileName00 = "Zone.csv"    # 地点データのファイル
FileName01 = "IncidentAngleCharacteristics.csv"   # 窓ガラスの入射角特性ファイル
ClimateZone0 = range(1,9)            # 地域区分(1～8地域, 他、ユニークなID設定可)
NDT = 6                    # 1時間の分割数,ツールの標準は6分割
Azimuth0 = ["北","北東","東","南東","南","南西","西","北西"]
# Azimuth0 = ['N','NE','E','SE','S','SW','W','NW']
    # 窓面の方位を16方位か、角度(-180°< Azimuth <= 180°)で入力
etaID = 1                  # 入射角特性のデータセットのID
WSSize = [999, 1, 999, 999, 999, 999, 999, 0.5, 2.1, 0, 0.5, 0.5, 0, 0, 0, 0, 2, 0]
    # WSSize = [X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym]
    # WSSize:窓および日よけの寸法一式


for ClimateZone in ClimateZone0: 
    for Azimuth in Azimuth0: 
        SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                                              , ClimateZone, NDT, etaID, Azimuth, WSSize )
                 
        # SCF[h][i][j]
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,O
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算             

        [SCFh, SCFc] = [SCF[2][-1][-1], SCF[2][-2][-1]]
        print('{}地域庇2m{}: SCFh = {}, SCFc = {}'.format( ClimateZone, Azimuth, SCFh, SCFc ))


Path00 = "./SCFConfig01/"  # 設定ファイルのあるパス
FileName00 = "Zone.csv"    # 地点データのファイル
FileName01 = "IncidentAngleCharacteristics.csv"   # 窓ガラスの入射角特性ファイル
ClimateZone = 8            # 地域区分(1～8地域, 他、ユニークなID設定可)
NDT = 6                    # 1時間の分割数,ツールの標準は6分割
Azimuth = '南'             # 窓面の方位を16方位か、角度(-180°< Azimuth <= 180°)で入力
etaID = 1                  # 入射角特性のデータセットのID
WSSize = [999, 1, 999, 999, 999, 999, 999, 0.5, 2.1, 0, 0.5, 0.5, 0, 0, 0, 0, 0, 0]
    # WSSize = [X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym]
    # WSSize:窓および日よけの寸法一式

SCF = SCFMain.Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01
                                           , ClimateZone, NDT, etaID, Azimuth, WSSize )
                 
    # SCF[h][i][j]
    # h = 0：分子, h = 1：分母, h = 2：効果係数
    # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,O
    #         i = 1～12：各月で積算
    # j = -1：日積算, j = 0～24：各時刻で積算             

[SCFh, SCFc] = [SCF[2][-1][-1], SCF[2][-2][-1]]
print('8地域庇なし: SCFh = {}, SCFc = {}'.format( SCFh, SCFc ))


1地域庇2m北: SCFh = 0.6693335123582321, SCFc = 0.6475770019718339
1地域庇2m北東: SCFh = 0.6537255862016472, SCFc = 0.623437291022517
1地域庇2m東: SCFh = 0.6391288121624638, SCFc = 0.6013791187592611
1地域庇2m南東: SCFh = 0.6271949916251893, SCFc = 0.5152123989865749
1地域庇2m南: SCFh = 0.6155078566444951, SCFc = 0.4509718460933475
1地域庇2m南西: SCFh = 0.6252518087634689, SCFc = 0.5086273444686046
1地域庇2m西: SCFh = 0.6383010761155143, SCFc = 0.5992342465289927
1地域庇2m北西: SCFh = 0.6539039252230086, SCFc = 0.6232306854609287
2地域庇2m北: SCFh = 0.6634271019758498, SCFc = 0.6496280549349986
2地域庇2m北東: SCFh = 0.6466583780317264, SCFc = 0.623469614246316
2地域庇2m東: SCFh = 0.6372951048575446, SCFc = 0.5974140023077601
2地域庇2m南東: SCFh = 0.6174415991849425, SCFc = 0.5099165666187745
2地域庇2m南: SCFh = 0.5795251350347503, SCFc = 0.44641225025302905
2地域庇2m南西: SCFh = 0.6068208633387927, SCFc = 0.5030785992729769
2地域庇2m西: SCFh = 0.6366231813371119, SCFc = 0.5983856883078693
2地域庇2m北西: SCFh = 0.6474364142510834, SCFc = 0.6245635046305986
3